# Import and setup

In [1]:
import os
import random
import numpy as np
import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset

# Librerie personalizzate
from dataset import BloodCellDataset
from model import CustomModel
from train import train_one_epoch, validate
from utils import plot_confusion_matrix, plot_normalized_confusion_matrix

# Albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

c:\Users\Mauro\anaconda3\envs\my_ai_env\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


 # Parameters 
 Initialization of some hyperparameters.

In [ ]:
DATASET_PATH = "" # Path to the dataset
VALID_FOLDERS = [] # To select only some folders from the dataset
IMG_SIZE = 400 # Image size: height = width = IMG_SIZE

TEST_RATIO = 0.1  # 10% set test
N_FOLDS = 5
SEED = 42 # Fixed seed value

BATCH_SIZE = 32
EPOCHS = 20 
LR = 1e-4 # Learning rate
MODEL_NAME = "tf_efficientnetv2_b0" # Choose model from timm library. For example: "tf_efficientnetv2_b0" or "resnet18"
PRETRAINED = False # Use True if you want to use a pretrained model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device:", DEVICE)
# Set all the seeds for reproducibility
random.seed(SEED) # set seed for random module
np.random.seed(SEED) # set seed for numpy module
torch.manual_seed(SEED) # set seed for torch module

# Dataset Creation
In this section dataset is created and loaded.

In [ ]:
full_dataset = BloodCellDataset(
    root_dir=DATASET_PATH,
    valid_folders=VALID_FOLDERS,
    img_size=IMG_SIZE,
    transform=None  # No transformations
)

all_labels = np.array(full_dataset.labels) # Return all the labels in the dataset
all_indices = np.arange(len(full_dataset)) # Return the indices of all the examples in the dataset

# Dataset fragmentation
A shuffle of all the indexes of the dataset is performed, for each class, and creates the test and train set.


In [7]:
class_to_indices = {} 
# Group the indices of the examples by class and put them in a dictionary class_to_indices
for idx, label in enumerate(all_labels):
    class_to_indices.setdefault(label, []).append(idx)

test_indices = [] # List of indices for the test set
trainval_indices = [] # List of indices for the train+val set (for K-fold cross-validation)

for label, idx_list in class_to_indices.items():
    idx_arr = np.array(idx_list) # Convert the list of indices to a numpy array
    np.random.shuffle(idx_arr)  # Shuffle the indices
    n_test = int(len(idx_arr)*TEST_RATIO) # Number of test examples for the current class

    test_part = idx_arr[:n_test] # Take n_test random indices for the test set
    train_part = idx_arr[n_test:]  # Take the remaining indices for the train set

    test_indices.extend(test_part) # Add the test indices to the test set
    trainval_indices.extend(train_part) # Add the train indices to the train set

test_indices = np.array(test_indices)
trainval_indices = np.array(trainval_indices)

print(f"Test set esterno: {len(test_indices)} esempi")
print(f"Train+Val set (K-fold): {len(trainval_indices)} esempi")

final_test_dataset = Subset(full_dataset, test_indices) # Test set 

Test set esterno: 3298 esempi
Train+Val set (K-fold): 29720 esempi


# K-FOLD
Il dataset composto dal 90% degli esempi (stratificato per classe) viene impiegato per il K-Fold cross validation.

In [9]:
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
labels_for_90 = all_labels[trainval_indices]

# Data augmentation per train
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=180, p=0.5),
    A.RandomBrightnessContrast(p=0.1),
    A.Resize(IMG_SIZE,IMG_SIZE),
    ToTensorV2()
])

# Transform di base per val/test
val_transform = A.Compose([
    A.Resize(IMG_SIZE,IMG_SIZE),
    ToTensorV2()
])

# Per la WeightedRandomSampler, calcoliamo le class weights
# 1 / freq_di_classe = peso. (Es. meno frequente = peso maggiore)
unique_labels, counts = np.unique(labels_for_90, return_counts=True)
class_counts = dict(zip(unique_labels, counts))

# Ad es. weight[class] = 1/count
max_count = max(class_counts.values())
weight_per_class = {}
for c, cnt in class_counts.items():
    weight_per_class[c] = max_count / cnt  # un modo per "bilanciare"

def make_sampler(indices_array):
    # Ritorna un WeightedRandomSampler per oversampling
    # in base a weight_per_class.
    sample_weights = []
    for idx in indices_array:
        label = all_labels[idx]
        sample_weights.append(weight_per_class[label])
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(indices_array),  # parità con dimensione set
        replacement=True  # sampling con rimpiazzo
    )
    return sampler

# Funzione per creare un dataset con transform
class SubsetAlbumentations(Dataset):
    def __init__(self, base_dataset, indices, albumentations_transform=None):
        self.base_dataset = base_dataset
        self.indices = indices
        self.transform = albumentations_transform
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, i):
        real_idx = self.indices[i]
        img, label = self.base_dataset[real_idx]
        # Se dataset ritorna un np array HxWxC o un Tensor CxHxW...
        # Adattiamo per Albumentations
        if isinstance(img, torch.Tensor):
            img_np = img.permute(1,2,0).numpy()
        else:
            img_np = img
        if self.transform:
            augmented = self.transform(image=img_np)
            img = augmented["image"]
        return img, label

fold_accuracies = []
fold_number = 1

# K FOLD EXEC

In [ ]:
for train_idx, val_idx in skf.split(trainval_indices, labels_for_90):
    print(f"\n=== Fold {fold_number}/{N_FOLDS} ===")
    real_train_indices = trainval_indices[train_idx]
    real_val_indices   = trainval_indices[val_idx]

    print("Train set fold:", len(real_train_indices), "| Val set fold:", len(real_val_indices))

    ds_train = SubsetAlbumentations(full_dataset, real_train_indices, albumentations_transform=train_transform)
    ds_val   = SubsetAlbumentations(full_dataset, real_val_indices, albumentations_transform=val_transform)

    sampler_train = make_sampler(real_train_indices)
    dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, sampler=sampler_train, num_workers=0)
    dl_val   = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = CustomModel(
        num_classes=full_dataset.num_classes,
        model_name=MODEL_NAME,
        pretrained=PRETRAINED
    ).to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    best_val_acc = 0.0

    for epoch in range(EPOCHS):
        print(f"  Epoch {epoch+1}/{EPOCHS}")
        train_loss, train_acc = train_one_epoch(model, dl_train, criterion, optimizer, DEVICE)
        val_loss, val_acc, precision, recall, f1, preds, labels = validate(model, dl_val, criterion, DEVICE)

        print(f"    TrainLoss: {train_loss:.4f}  Acc: {train_acc:.4f}"
              f" | ValLoss: {val_loss:.4f}  Acc: {val_acc:.4f}")

        # --- SALVATAGGIO DEL BEST MODEL ---
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"{MODEL_NAME}_best_fold{fold_number}.pth")
            print(f"    ► Salvo best model per fold {fold_number} con val_acc = {best_val_acc:.4f}")

    fold_accuracies.append(best_val_acc)
    print(f"*** Best Val Acc Fold {fold_number}: {best_val_acc:.4f} ***")

    fold_number += 1


# Risultati k-cross

In [ ]:
mean_acc = np.mean(fold_accuracies)
std_acc = np.std(fold_accuracies)
print(f"\nRISULTATI CROSS VALIDATION: acc media {mean_acc:.4f} +/- {std_acc:.4f}")


# Training su tutto il 90% e test

In [10]:
ds_train_90 = SubsetAlbumentations(full_dataset, trainval_indices, albumentations_transform=train_transform)
sampler_90  = make_sampler(trainval_indices)
dl_train_90 = DataLoader(ds_train_90, batch_size=BATCH_SIZE, sampler=sampler_90, num_workers=0)

ds_test = SubsetAlbumentations(full_dataset, test_indices, albumentations_transform=val_transform)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False)

final_model = CustomModel(num_classes=full_dataset.num_classes, model_name=MODEL_NAME, pretrained=PRETRAINED).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=LR)

In [ ]:
print("\n=== TRAINING FINALE SU TUTTO IL 90% ===")
EPOCHS = 30
best_val_acc = 0.0
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(final_model, dl_train_90, criterion, optimizer, DEVICE)
    val_loss, val_acc, precision, recall, f1, preds, labels = validate(final_model, dl_test, criterion, DEVICE)

    print(f"    TrainLoss: {train_loss:.4f}  Acc: {train_acc:.4f}"
              f" | ValLoss: {val_loss:.4f}  Acc: {val_acc:.4f}")

    # --- SALVATAGGIO DEL BEST MODEL ---
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(final_model.state_dict(), f"{MODEL_NAME}_best_90.pth")
        print(f"    ► Salvo best model per fold {fold_number} con val_acc = {best_val_acc:.4f}")

torch.save(final_model.state_dict(), f"{MODEL_NAME}_final_90perc.pth")
print("► Salvato il modello addestrato su tutto il 90%.")

# Valutazione

In [ ]:
# 1) Crea un’istanza del modello, identico a quello usato in training
loaded_model = CustomModel(
    num_classes=full_dataset.num_classes,
    model_name=MODEL_NAME, 
    pretrained=False
).to(DEVICE)

# 2) Carica i pesi salvati
loaded_model.load_state_dict(torch.load("tf_efficientnetv2_b0_best_90.pth"))

# 3) Metti il modello in eval mode
loaded_model.eval()

# 4) Valuta sul test set
test_loss, test_acc, p, r, f1, preds, labels = validate(
    loaded_model, 
    dl_test,      # il DataLoader del test
    criterion,    # ad esempio nn.CrossEntropyLoss()
    DEVICE
)

print("=== EVALUATION DEL MODELLO CARICATO ===")
print(f"Test Loss {test_loss:.4f}, Test Acc {test_acc:.4f}")
print(f"Precision {p:.2f}, Recall {r:.2f}, F1 {f1:.2f}")

plot_confusion_matrix(labels, preds, full_dataset.classes)
plot_normalized_confusion_matrix(labels, preds, full_dataset.classes)
